In [1]:
import pandas as pd
df= pd.read_csv("vitals.csv")

In [2]:
df

,first_name,last_name,gender,dob,blood_grp,marital_stat,height,weight,emergency_contact,address,...,Step,Step_Measured_At,Height,Height_Measured_At,Stethocope,Stethocope_Measured_At,Otoscope,Otoscope_Measured_At,ECG,ECG_Measured_At
0,Tapan,R,1,23/10/93,O+,1,125.0,65.0,7.757534e+09,"pune, maharastara",...,45.0,10/30/2024 4:42:02 PM,181.0,10/12/24 13:14,NaN,10/18/2024 6:52:04 PM,NaN,10/21/2024 6:23:35 PM,NaN,10/18/2024 10:53:52 AM
1,Abbas,Mulani,1,01/06/88,NaN,0,NaN,NaN,NaN,Pune India,...,65.0,10/30/2024 11:11:30 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/30/2024 11:15:53 AM
2,Sam,Richard,1,17/09/93,NaN,0,NaN,NaN,NaN,NaN,...,822.0,10/15/2024 11:03:25 PM,178.0,2/16/2024 12:02:25 PM,NaN,1/27/2024 6:50:11 PM,NaN,NaN,NaN,02/05/24 13:01
3,Om,Puri,1,13/06/00,NaN,0,110.0,50.0,8.505159e+09,Pune,...,5941.0,6/21/2021 9:44:57 PM,157.0,10/11/24 17:51,NaN,NaN,NaN,NaN,NaN,6/21/2021 8:22:20 PM
4,Samir,Ranavaya,1,17/09/93,B+,0,NaN,NaN,NaN,NaN,...,32.0,10/01/24 15:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Test,member,0,02/02/00,B+,0,154.0,60.0,NaN,NaN,...,24.0,9/13/2024 9:55:12 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/18/2024 2:51:23 PM
6,Jayashree,V,0,02/08/93,NaN,0,NaN,NaN,NaN,NaN,...,109.0,08/05/24 10:38,154.0,8/26/2024 6:11:48 PM,NaN,NaN,NaN,NaN,NaN,NaN
7,Inamdar,hospital,1,26/08/89,NaN,0,NaN,NaN,NaN,NaN,...,223.0,8/26/2024 6:28:21 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Anuradha,rai,0,07/04/93,NaN,0,NaN,NaN,NaN,NaN,...,899.0,08/09/24 15:52,NaN,NaN,NaN,6/28/2024 3:04:18 PM,NaN,6/28/2024 3:00:30 PM,NaN,NaN
9,Suraj,Joshi,1,08/04/99,B+,0,NaN,NaN,NaN,NaN,...,37.0,08/08/24 15:20,172.0,04/08/24 17:59,NaN,04/08/24 17:57,NaN,04/08/24 17:55,NaN,04/08/24 17:50


In [4]:
from flask import Flask, render_template, request
from flask_socketio import SocketIO, send
import pandas as pd
from datetime import datetime

# Load the data
file_path = "vitals.csv"
data = pd.read_csv(file_path)
data['dob'] = pd.to_datetime(data['dob'], errors='coerce')
data['age'] = data['dob'].apply(lambda dob: datetime.today().year - dob.year if pd.notnull(dob) else None)

# Initialize Flask app and Flask-SocketIO
app = Flask(__name__)
app.config['SECRET_KEY'] = 'mysecret'
socketio = SocketIO(app)

# Define the query functions as in the previous solution
def max_age_patient():
    max_age = data['age'].max()
    max_age_patient = data[data['age'] == max_age][['first_name', 'last_name', 'age']]
    return max_age_patient.to_dict(orient='records')

def highest_blood_pressure():
    max_bp = data['BPSystolic'].max()
    highest_bp_patient = data[data['BPSystolic'] == max_bp][['first_name', 'last_name', 'BPSystolic', 'BPSystolic_Measured_At']]
    return highest_bp_patient.to_dict(orient='records')

def patients_by_bp_desc():
    sorted_bp_data = data.sort_values(by='BPSystolic', ascending=False)[['first_name', 'last_name', 'BPSystolic']]
    return sorted_bp_data.to_dict(orient='records')

# Function to handle queries
def answer_query(query):
    if "max age" in query.lower():
        return max_age_patient()
    elif "highest blood pressure" in query.lower():
        return highest_blood_pressure()
    elif "patients by blood pressure" in query.lower() or "blood pressure descending" in query.lower():
        return patients_by_bp_desc()
    else:
        return "I didn't understand that question. Please try again."

# Define a route for the chatbot UI
@app.route('/')
def home():
    return render_template('chat.html')

# Handle messages from the client
@socketio.on('message')
def handle_message(msg):
    response = answer_query(msg)
    send(response, broadcast=True)

# Run the Flask app
if __name__ == '__main__':
    socketio.run(app, debug=True)


[2024-11-10 17:51:28,503] WARNING in __init__: WebSocket transport not available. Install gevent-websocket for improved performance.
 * Restarting with windowsapi reloader


SystemExit: 1

C:\Users\Owner\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
C:\Users\Owner\anaconda3\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
